# Redis Vector Search and RedisVL Demo

This notebook demonstrates how to use Redis as a vector database and how to simplify vector search using the RedisVL library.

In [ ]:
import redis
import numpy as np
import json

r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Function to create a dummy vector with 8 dimensions
def create_vector():
    return np.round(np.random.rand(8), 3).tolist()

# Insert dummy vector records
for i in range(5):
    doc = {
        "id": f"vec{i}",
        "content": f"This is sample vector document {i}",
        "embedding": create_vector()
    }
    r.json().set(f"vec:{i}", "$", doc)

print("Inserted vector documents.")

In [ ]:
# Create RediSearch vector index
from redis.commands.search.field import TextField, VectorField

try:
    r.ft("idx:vecs").dropindex(delete_documents=False)
except:
    pass

schema = [
    TextField("$.content", as_name="content"),
    VectorField("$.embedding", "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": 8,
        "DISTANCE_METRIC": "COSINE",
        "INITIAL_CAP": 100
    }, as_name="embedding")
]

r.ft("idx:vecs").create_index(schema, on="JSON", prefix=["vec:"])
print("Vector index created.")

In [ ]:
# Vector KNN query
from redis.commands.search.query import Query
import struct

query_vector = create_vector()
blob = np.array(query_vector, dtype=np.float32).tobytes()

q = Query("*=>[KNN 2 @embedding $vec AS score]").return_fields("id", "score").sort_by("score").dialect(2)
res = r.ft("idx:vecs").search(q, query_params={"vec": blob})

for doc in res.docs:
    print(doc.id, doc.score)

## RedisVL Library Demo

In [ ]:
!pip install redisvl
from redisvl import Client, Index
from redisvl.schema import IndexSchema, TextField, VectorField

# Connect
client = Client()

# Define schema
schema = IndexSchema(
    TextField(name="content"),
    VectorField(name="embedding", dims=8, algorithm="flat", distance_metric="cosine")
)

# Create index
index = Index(name="idx:redisvl", prefix="vecvl:", schema=schema)
client.create_index(index)

# Insert sample doc
client.load_documents(
    documents=[
        {"id": "vecvl:1", "content": "vector doc 1", "embedding": create_vector()}
    ]
)

# Query
results = client.query(index.name).knn(vector=create_vector(), num=1).return_fields(["content", "embedding"]).execute()
print(results)